In [1]:
import numpy as np 
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import mean_squared_error

# 데이터 불러오기
data = pd.read_pickle('/Users/yul/Desktop/LP Player/3. LP Player_데이터 및 모델 세이브 파일/User-based_filtering_data_pickle_기본')
np.random.seed(19670324)  # seed 값 지정

# pivot_table 만들기

In [2]:
ratings = data.pivot_table(index='cust', columns='pd_nm', values='buy_ct')
ratings.fillna(0, inplace=True)  # na값 0으로 대체
ratings

pd_nm,2단우산,3단우산,BB/파운데이션/컴팩트류,CDP/카세트,DIY가구,DIY완구,LED,LED램프,MP3,PC부품,...,후크류,훈련,훈제/구이오징어,훈제/로스트,휴대폰,휴대폰용품,휴지통,흑미,히터,힙색/사이드백
cust,,,,,,,,,,,,,,,,,,,,,
M000225114,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
M000261625,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
M000419293,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
M000494848,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
M000705571,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
M997283992,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
M997831586,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
M998680257,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# 훈련 데이터와 테스트 데이터 분리

In [3]:
ratings_train, ratings_test = train_test_split(ratings, test_size=0.25, random_state=42)
ratings_train.shape, ratings_test.shape

((4278, 1836), (1427, 1836))

# 코사인 유사도

In [4]:
user_similarity = cosine_similarity(ratings_train)   # ratings_train의 사용자 유사도
user_similarity

array([[1.        , 0.        , 0.03504998, ..., 0.09733703, 0.06746068,
        0.        ],
       [0.        , 1.        , 0.        , ..., 0.0932516 , 0.01020461,
        0.33615618],
       [0.03504998, 0.        , 1.        , ..., 0.03005511, 0.02499609,
        0.        ],
       ...,
       [0.09733703, 0.0932516 , 0.03005511, ..., 1.        , 0.33220735,
        0.07846481],
       [0.06746068, 0.01020461, 0.02499609, ..., 0.33220735, 1.        ,
        0.01997671],
       [0.        , 0.33615618, 0.        , ..., 0.07846481, 0.01997671,
        1.        ]])

# 예측값 계산

In [5]:
# 예측값 계산
user_pred = user_similarity.dot(ratings_train) / np.array([np.abs(user_similarity).sum(axis=1)]).T
user_pred_matrix = pd.DataFrame(data=user_pred, index=ratings_train.index, columns=ratings_train.columns)
user_pred_matrix.head(10)

pd_nm,2단우산,3단우산,BB/파운데이션/컴팩트류,CDP/카세트,DIY가구,DIY완구,LED,LED램프,MP3,PC부품,...,후크류,훈련,훈제/구이오징어,훈제/로스트,휴대폰,휴대폰용품,휴지통,흑미,히터,힙색/사이드백
cust,,,,,,,,,,,,,,,,,,,,,
M062406215,0.000102,0.011564,0.234798,0.000000,0.000945,0.056005,0.012547,0.026434,0.000000,0.003075,...,0.012693,0.003005,0.095634,0.071329,0.031130,0.004739,0.025166,0.006282,0.000292,0.000105
M285142090,0.001034,0.010815,0.179779,0.000065,0.002184,0.093474,0.013323,0.064204,0.000000,0.000867,...,0.026248,0.002142,0.219483,0.309695,0.025688,0.014506,0.093923,0.035672,0.000434,0.000887
M149739353,0.000988,0.006028,0.237979,0.000870,0.000220,0.112474,0.004247,0.038954,0.000000,0.000958,...,0.008318,0.000554,0.078951,0.074364,0.035834,0.013501,0.030672,0.008404,0.000013,0.000403
M259385626,0.000793,0.011504,0.206987,0.000042,0.001571,0.093371,0.011499,0.048197,0.000130,0.001364,...,0.021167,0.002954,0.183447,0.246593,0.033244,0.012603,0.110620,0.024710,0.000664,0.000666
M637265077,0.000722,0.015742,0.163965,0.000000,0.001349,0.078469,0.009425,0.057305,0.000000,0.001595,...,0.023034,0.002078,0.232526,0.450400,0.036538,0.014362,0.125542,0.033744,0.000688,0.001062
M750158653,0.000868,0.009146,0.171398,0.000209,0.002238,0.100140,0.009648,0.053850,0.000046,0.000509,...,0.019729,0.001878,0.186753,0.198619,0.028704,0.014136,0.062194,0.025200,0.000442,0.000885
M062598719,0.001145,0.013084,0.156827,0.000038,0.001801,0.090414,0.007161,0.039946,0.000038,0.000916,...,0.018214,0.001058,0.215225,0.241516,0.036669,0.013361,0.066086,0.025008,0.000863,0.000847
M508302832,0.000913,0.012791,0.173484,0.000268,0.001023,0.098322,0.008663,0.060995,0.000000,0.000853,...,0.022122,0.002840,0.236284,0.328447,0.035077,0.018690,0.087435,0.033277,0.000604,0.001075
M225832472,0.000428,0.007528,0.199061,0.000385,0.001108,0.101945,0.007082,0.042064,0.000000,0.000441,...,0.014081,0.002137,0.165117,0.115083,0.035169,0.015662,0.045310,0.014716,0.000591,0.000647


# 유사한 고객 찾기

In [6]:
# user_similarity(사용자 유사도)는 array형태로 열 이름이 없기 때문에 DataFrame으로 바꿔주고 유사도 높은 고객을 찾아냄
user_similarity_df = pd.DataFrame(
    data=user_similarity,
    index=ratings_train.index,
    columns=ratings_train.index)

# 가장 유사도가 높은 top5
def recommand_user(cust):
    return user_similarity_df[cust].sort_values(ascending=False)[:6]

In [7]:
recom_user = recommand_user('M999227380')
recom_user

cust
M999227380    1.000000
M267665211    0.881097
M360187347    0.841298
M369057001    0.798437
M967393794    0.797352
M830253345    0.768678
Name: M999227380, dtype: float64

# 새로운 상품 추천

In [8]:
def no_buy(ratings_trian, user):
    user_rating = ratings_train.loc[user,:] # user 별로 평점 정보
    no_buy = user_rating[user_rating==0].index.tolist() # 사지 않은 상품 리스트
    product_list = ratings_train.columns.tolist() # 전체 상품 리스트
    no_list = [product for product in product_list if product in no_buy]
    return no_list

def recommend_product_by_user(pred_df, user, no_list, top_n=5):
    recommend_product = pred_df.loc[user, no_list].sort_values(ascending=False)[:top_n]
    return recommend_product

In [9]:
cust = recom_user.index[0]
no_buy = no_buy(ratings_train, cust)

recommend_product = recommend_product_by_user(user_pred_matrix, cust, no_buy, top_n=5)

recom_products = pd.DataFrame(data=recommend_product.values, index=recommend_product.index, columns=['pred_score'])
recom_products

,pred_score
pd_nm,
종량제봉투,5.827541
일반스낵,5.373377
기타파티/팬시용품,3.893924
감자스낵,3.550657
국물봉지라면,3.006956


# 모델 평가 지표 : RMSE

In [10]:
# RMSE는 DataFrame으로 계산 불가 -> array로 바꿔줌
ratings_train_np = np.array(ratings_train)
ratings_test_np = np.array(ratings_test)

# RMSE 함수
def get_rmse(pred, actual):
    pred = pred[actual.nonzero()].flatten()
    actual = actual[actual.nonzero()].flatten()
    return mean_squared_error(pred, actual)**0.5 # RMSE

In [11]:
print("train:", get_rmse(user_pred, ratings_train_np))
print("test:", get_rmse(user_pred, ratings_test_np))

train: 7.381880236477308
test: 8.246900453482464


# 군집 분석을 이용한 알고리즘 개선
- k-NN을 이용하여 가장 비슷한 몇 명의 사용자끼만 비교하도록 개선

In [12]:
k=5
neigh = NearestNeighbors(n_neighbors=k, metric="cosine")
neigh.fit(ratings_train)
top_k_distances, top_k_users = neigh.kneighbors(ratings_train, return_distance=True)
top_k_similarity = 1 - top_k_distances
top_k_similarity

array([[1.        , 0.60169465, 0.5834283 , 0.5817434 , 0.52849871],
       [1.        , 0.56942018, 0.55761258, 0.5551452 , 0.54778909],
       [1.        , 0.66950909, 0.66469405, 0.63960215, 0.63960215],
       ...,
       [1.        , 0.57946834, 0.55714559, 0.55191343, 0.55117609],
       [1.        , 0.51582887, 0.44365918, 0.44307357, 0.44055431],
       [1.        , 0.73214186, 0.6583685 , 0.64493017, 0.64477547]])

# 군집 분석 후 RMSE 평가

In [13]:
user_pred_k = np.zeros(ratings_train_np.shape)

for i in range(ratings_train_np.shape[0]):
    user_pred_k[i, :] = top_k_similarity[i].T.dot(ratings_train_np[top_k_users][i]) / np.array([np.abs(top_k_similarity[i].T).sum(axis=0)]).T

In [14]:
print("train:",get_rmse(user_pred_k, ratings_train_np))  # 훈련에 대한 평가지표가 개선됨
print("Test:",get_rmse(user_pred_k, ratings_test_np))

train: 6.065427735144704
Test: 9.422298969132864
